In [0]:
  from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
from tensorflow.python.keras import backend as K 
#K.set_learning_phase(0)
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import Model 
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras import optimizers

In [0]:
#!mkdir FinalData/
#!cp /content/drive/My\ Drive/Competitive/Dataset/ExtraCredit.zip FinalData/
#!unzip FinalData/ExtraCredit.zip -d FinalData/

#!mkdir ValData/
#!cp  /content/drive/My\ Drive/Competitive/Dataset/TestData.zip ValData/
#!unzip ValData/TestData.zip -d ValData/


train_data_dir = "/content/FinalData/ExtraCredit/Train"
val_data_dir = "/content/FinalData/ExtraCredit/Val"
num_train = 0
num_val = 0

for i in os.listdir(train_data_dir):
  if i.startswith("."):
    os.rmdir(os.path.join(train_data_dir, i))
  else:
    print(i,len(os.listdir(os.path.join(train_data_dir, i))))
    num_train += len(os.listdir(os.path.join(train_data_dir, i)))
  
for i in os.listdir(val_data_dir):
  if i.startswith("."):
    os.rmdir(os.path.join(val_data_dir, i))
  else:
    num_val += len(os.listdir(os.path.join(val_data_dir, i)))

img_width, img_height = 224, 224
# Check if the images are RGB and change thefrom tensorflow.python.keras.layers import Dense

if K.image_data_format() == 'channels_first':
  input_shape= (3, img_width, img_height)
else:
  input_shape = (img_width, img_height, 3)

3 36
4 67
1 149
0 338
2 30


In [0]:
#from tensorflow.python.keras.layers import Dense
#from tensorflow.python.keras import Sequential
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.python.keras.applications.densenet import DenseNet121
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.vgg19 import VGG19
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet101, ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.xception import Xception


base_model = ResNet50(input_shape=(224, 224, 3),
                                  weights='imagenet',
                                  include_top = False,
                                  pooling='avg')

for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = Dense(1024)(x)
x = Activation('relu')(x)
x = Dense(512)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Dropout(0.1)(x) 
x = Activation('relu')(x)

predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001), metrics = ['accuracy'])
#model.summary()
model.load_weights("FinalData/ResNet50ExtraCredit.h5")


In [0]:
import pandas as pd
from sklearn.metrics import classification_report
df_train = pd.read_csv("/content/ExtraCredit_Train.csv")
df_test = pd.read_csv("/content/ExtraCredit_Test.csv")
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier


df_train_0 = df_train[df_train.annotation == 0]
df_train_1 = df_train[df_train.annotation == 1]
df_train_2 = df_train[df_train.annotation == 2]
df_train_3 = df_train[df_train.annotation == 3]
df_train_4 = df_train[df_train.annotation == 4]

df_train_1 = resample(df_train_1, replace=True, n_samples=338, random_state=123)
df_train_2 = resample(df_train_2, replace=True, n_samples=338, random_state=123)
df_train_3 = resample(df_train_3, replace=True, n_samples=338, random_state=123)
df_train_4 = resample(df_train_4, replace=True, n_samples=338, random_state=123)

df_train_bal = pd.concat([df_train_0, df_train_1, df_train_2, df_train_3, df_train_4])
y_train = df_train_bal['annotation']
X_train = df_train_bal.iloc[:,2:]

y_test = df_test['annotation']
X_test = df_test.iloc[:,2:]

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)

clf = ExtraTreesClassifier(n_estimators=50).fit(X_train, y_train)
model = SelectFromModel(clf, prefit=True)
X_new_train = model.transform(X_train)
X_new_test = model.transform(X_test)

print(X_new_train.shape)
print(X_new_test.shape)
print(df_train_bal.annotation.value_counts())
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
ans = clf.fit(X_new_train, y_train)
y_pred = clf.predict(X_new_test)
print(classification_report(y_test, y_pred))

(1690, 12)
(1690, 4)
(100, 4)
4    338
3    338
2    338
1    338
0    338
Name: annotation, dtype: int64
              precision    recall  f1-score   support

           0       0.55      0.80      0.65        20
           1       0.79      0.75      0.77        20
           2       0.84      0.80      0.82        20
           3       0.71      0.50      0.59        20
           4       0.74      0.70      0.72        20

    accuracy                           0.71       100
   macro avg       0.73      0.71      0.71       100
weighted avg       0.73      0.71      0.71       100



In [0]:
print(y_test.to_xarray)
prob = clf.predict_proba(X_new_test)


<bound method NDFrame.to_xarray of 0     2
1     2
2     2
3     4
4     4
     ..
95    1
96    1
97    1
98    1
99    1
Name: annotation, Length: 100, dtype: int64>


In [0]:
import numpy
import sklearn.metrics as metrics
img_width, img_height = 224, 224
batch_size =1 
epochs = 100
test_set_dir = "/content/drive/My Drive/Competitive/Extra/Val"
val_set_dir = "/content/drive/My Drive/Competitive/Extra/Val"

num_test = len(os.listdir(test_set_dir))

print ("Number of images in test set: ", num_test)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(val_set_dir, 
                                                    target_size =(img_width, img_height), 
                                                    batch_size = batch_size, class_mode = 'categorical',color_mode='rgb',shuffle=False)
 
test_steps_per_epoch = numpy.math.ceil(test_generator.samples / test_generator.batch_size)

predictions = model.predict_generator(test_generator, steps = test_generator.samples)
#print(predictions)
predicted_classes = numpy.argmax(predictions, axis=1)

files = test_generator.filenames
def validation():
  names = []
  for n in files:
      names.append(int(n.split("_")[1].split(".")[0]))
  acc = 0
  print(metrics.confusion_matrix(names,predicted_classes))
  print(metrics.accuracy_score(names,predicted_classes))
  print(metrics.classification_report(names,predicted_classes))
  print(names)
validation()


Number of images in test set:  1
Found 100 images belonging to 1 classes.
[[18  2  0  0  0]
 [ 0 19  1  0  0]
 [ 3  0  7 10  0]
 [ 0  0  2 17  1]
 [ 0  0  0  6 14]]
0.75
              precision    recall  f1-score   support

           0       0.86      0.90      0.88        20
           1       0.90      0.95      0.93        20
           2       0.70      0.35      0.47        20
           3       0.52      0.85      0.64        20
           4       0.93      0.70      0.80        20

    accuracy                           0.75       100
   macro avg       0.78      0.75      0.74       100
weighted avg       0.78      0.75      0.74       100

[2, 2, 2, 4, 4, 2, 2, 2, 4, 3, 2, 3, 4, 4, 4, 2, 2, 4, 2, 3, 4, 4, 4, 3, 4, 3, 3, 3, 4, 3, 3, 4, 4, 3, 2, 3, 3, 3, 3, 2, 2, 3, 4, 2, 2, 4, 3, 2, 2, 2, 3, 0, 0, 0, 4, 0, 4, 0, 3, 0, 0, 3, 2, 0, 4, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [0]:
prob1=prob*0.5
predictions1=predictions*0.5
new_score=np.add(prob1,predictions1)
predicted_classes = numpy.argmax(new_score, axis=1)

files = test_generator.filenames
def validation():
  names = []
  for n in files:
      names.append(int(n.split("_")[1].split(".")[0]))
  acc = 0
  print(metrics.confusion_matrix(names,predicted_classes))
  print(metrics.accuracy_score(names,predicted_classes))
  print(metrics.classification_report(names,predicted_classes))
  print(names)
validation()

[[19  1  0  0  0]
 [ 0 20  0  0  0]
 [ 3  0 14  3  0]
 [ 0  0  2 18  0]
 [ 0  0  0  6 14]]
0.85
              precision    recall  f1-score   support

           0       0.86      0.95      0.90        20
           1       0.95      1.00      0.98        20
           2       0.88      0.70      0.78        20
           3       0.67      0.90      0.77        20
           4       1.00      0.70      0.82        20

    accuracy                           0.85       100
   macro avg       0.87      0.85      0.85       100
weighted avg       0.87      0.85      0.85       100

[2, 2, 2, 4, 4, 2, 2, 2, 4, 3, 2, 3, 4, 4, 4, 2, 2, 4, 2, 3, 4, 4, 4, 3, 4, 3, 3, 3, 4, 3, 3, 4, 4, 3, 2, 3, 3, 3, 3, 2, 2, 3, 4, 2, 2, 4, 3, 2, 2, 2, 3, 0, 0, 0, 4, 0, 4, 0, 3, 0, 0, 3, 2, 0, 4, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
